In [ ]:
# 최신 ngrok v3 다운로드
!wget -q https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xvzf ngrok-v3-stable-linux-amd64.tgz


# ngrok 토큰 설정 (한 번만)
# 아래 링크에서 본인 토큰 발급: https://dashboard.ngrok.com/get-started/setup
!./ngrok authtoken <YOUR_NGROK_TOKEN>


In [ ]:
!pip install python-Levenshtein
!pip install speechrecognition

In [ ]:
from flask import Flask, request
import speech_recognition as sr
import tempfile
import os
import threading
import subprocess
import time
import requests
import Levenshtein

# ==================== 1. Flask 앱 정의 ====================
app = Flask(__name__)
recognizer = sr.Recognizer()

COMMANDS = {
    "전등켜": "light_on", "전등꺼": "light_off",
    "에어컨켜": "ac_on", "에어컨꺼": "ac_off",
    "커튼열어": "curtain_open", "커튼닫아": "curtain_close",
    "티비켜": "tv_on", "티비꺼": "tv_off", "다음채널" : "tv_channelUP", "이전채널" : "tv_channelDOWN",
    "볼륨올려" : "tv_volumeUP", "볼륨내려" : "tv_volumeDOWN"
}

# ==================== 2. 명령어 매칭 (유사도 기반) ====================
def match_command(text, threshold=3):
    text = text.replace(" ", "")  # 예: "전 등 켜" → "전등켜"
    best_match = None
    best_score = float('inf')

    for command in COMMANDS:
        dist = Levenshtein.distance(text, command)
        if dist < best_score and dist <= threshold:
            best_match = command
            best_score = dist

    return COMMANDS.get(best_match, None)

# ==================== 3. /infer 엔드포인트 (단순 텍스트 응답) ====================
@app.route('/infer', methods=['POST'])
def infer():
    if 'audio' not in request.files:
        return "ERROR|No audio file uploaded", 400

    file = request.files['audio']
    with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmp:
        file.save(tmp.name)
        tmp_path = tmp.name

    try:
        with sr.AudioFile(tmp_path) as source:
            audio = recognizer.record(source)

        recognized_text = recognizer.recognize_google(audio, language="ko-KR").strip()
        action = match_command(recognized_text) or "none"

        # 단순 텍스트 응답: "인식텍스트|액션"
        return f"{recognized_text}|{action}"

    except sr.UnknownValueError:
        return "ERROR|STT 인식 실패: 무음 또는 잡음", 400
    except Exception as e:
        return f"ERROR|{str(e)}", 500
    finally:
        if os.path.exists(tmp_path):
            os.remove(tmp_path)

# ==================== 4. Flask 서버 실행 (스레드) ====================
def run_flask():
    app.run(host="0.0.0.0", port=5000)

threading.Thread(target=run_flask).start()

# ==================== 5. ngrok 실행 ====================
time.sleep(2)
ngrok_process = subprocess.Popen(["./ngrok", "http", "5000"])
time.sleep(3)

# ==================== 6. ngrok 주소 얻기 ====================
def get_ngrok_url():
    try:
        resp = requests.get("http://localhost:4040/api/tunnels")
        return resp.json()['tunnels'][0]['public_url']
    except Exception as e:
        print("❌ ngrok URL 가져오기 실패:", e)
        return None

url = get_ngrok_url()
print("🔗 Colab 서버 주소:", url)

# ==================== 7. 무한 루프로 셀 유지 ====================
while True:
    time.sleep(1)